### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [ ]:
import numpy as np
import pandas as pd
import re
from datetime import datetime
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

from IPython.display import Markdown, display, HTML
def printmd(string):
    display(Markdown(string))


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

### DataFrame Head

In [2]:
display(movies.head())
printmd('### shape of movies' + str(movies.shape))

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


### shape of movies(35479, 3)

In [3]:
display(reviews.head())
printmd('### shape of reviews' + str(reviews.shape))

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


### shape of reviews(863866, 4)

In [4]:
movies.query('movie_id=="0114508"')

,movie_id,movie,genre
8883,0114508,Species (1995),Action|Horror|Sci-Fi|Thriller


### Descriptive Statistics

In [5]:
movies.describe(include='all').T

,count,unique,top,freq
movie_id,35479,35479,0396963,1
movie,35479,35416,Home (2016),4
genre,35230,2736,Drama,3602


In [6]:
reviews.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
user_id,863866,67353,23751,2875,NaN,NaN,NaN,NaN,NaN,NaN,NaN
movie_id,863866,35479,1454468,3086,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rating,863866,NaN,NaN,NaN,7.31588,1.85383,0,6,8,9,10
timestamp,863866,851107,1484827876,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### DataFrame Types

In [7]:
movies.dtypes

movie_id    object
movie       object
genre       object
dtype: object

In [8]:
reviews.dtypes

user_id      object
movie_id     object
rating        int64
timestamp    object
dtype: object

### DataFrame Overview

In [9]:
def get_dataframe_summary(df, cat_list, dummy_list):
    """ a reusable function for a dataframe summary in addition to the dataframe-describe() function
    
        INPUT: 
        ----------
        - df-  the DataFrame which should be described in an overview
        - cat_list - a list of all categorical input features
        - dummy_list - a list of all dummy input features 

        OUTPUT:
        ----------
        - print statements within the JUpyter notebook
        - output is auto generated
    """
    
    number_nan = df.isnull().sum()
    printmd('### DataFrame Overview')
    printmd('- Dataset with {} observations and {} columns'.format(df.shape[0], df.shape[1]))
    
    text_for_readme = '## DataFrame Overview\n'
    text_for_readme += 'Dataset with {} observations and {} columns\n\n'.format(df.shape[0], df.shape[1])
    
    print('___________________________________')
    printmd('- **Numerical** columns:')
    
    text_for_readme += '- **Numerical** columns:\n\n'
    text_for_readme += '\t| column_name | type | min | max | number NaN |\n\t| :-------------  | :-------------  | :-------------  | :-------------  | :-------------  |\n'
    row_num = []
    count_int = 0
    count_float = 0
    for col in df.columns.tolist():
        if (df[col].dtypes == 'float64' or df[col].dtypes == 'int64') and col not in cat_list and col not in dummy_list:
            row_num.append([col, df[col].dtypes, df[col].min(), df[col].max(), number_nan[col]])
            text_for_readme +=('\t| ' + str(col) + ' | ' + str(df[col].dtypes) + ' | ' + str(df[col].min()) + ' | ' + str(df[col].max()) + ' | '+ str(number_nan[col]) + ' | ' + '\n') 
            if df[col].dtypes == 'int64':
                count_int += 1
            if df[col].dtypes == 'float64':
                count_float += 1
    df_num = pd.DataFrame(row_num,columns=['column_name', 'type', 'min', 'max', 'number NaN'])
    text_for_readme +='\n\n'
    display(df_num)

    print('___________________________________')
    printmd('- **Categorical** columns:')
    
    text_for_readme += '- **Categorical** columns:\n\n'
    text_for_readme += '\t| column_name | type | min | max | number NaN |\n\t| :-------------  | :-------------  | :-------------  | :-------------  | :-------------  |\n'
    row_cat = []
    count_object = 0
    for col in df.columns.tolist():
        if df[col].dtypes == 'object' or col in cat_list:
            try:
                row_cat.append([col, df[col].dtypes, df[col].min(), df[col].max(), number_nan[col]])
                text_for_readme +=('\t| ' + str(col) + ' | ' + str(df[col].dtypes) + ' | ' + str(df[col].min()) + ' | ' + str(df[col].max()) + ' | '+ str(number_nan[col]) + ' | ' + '\n') 
            except:
                row_cat.append([col, df[col].dtypes, 'NaN', 'NaN', number_nan[col]])
                text_for_readme +=('\t| ' + str(col) + ' | ' + str(df[col].dtypes) + ' | '+ 'NaN' + ' | '+ 'NaN' + ' | '+ str(number_nan[col]) + ' | ' + '\n') 
            count_object += 1
    df_cat = pd.DataFrame(row_cat,columns=['column_name', 'type', 'min', 'max', 'number NaN'])
    text_for_readme +='\n\n'
    display(df_cat)
    
    print('___________________________________')
    printmd('- **Dummy** columns:')
    
    text_for_readme += '- **Dummy** columns:\n\n'
    text_for_readme += '\t| column_name | type | min | max | number NaN |\n\t| :-------------  | :-------------  | :-------------  | :-------------  | :-------------  |\n'
    row_dummy = []
    for col in dummy_list:
        if (df[col].dtypes == 'float64' or df[col].dtypes == 'int64'):
            row_dummy.append([col, df[col].dtypes, df[col].min(), df[col].max(), number_nan[col]])
            text_for_readme +=('\t| ' + str(col) + ' | ' + str(df[col].dtypes) + ' | ' + str(df[col].min()) + ' | ' + str(df[col].max()) + ' | '+ str(number_nan[col]) + ' | ' + '\n')
    df_dummy = pd.DataFrame(row_dummy,columns=['column_name', 'type', 'min', 'max', 'number NaN'])
    text_for_readme +='\n\n'
    display(df_dummy)
    
    print('___________________________________')
    printmd('- There are ***{} numerical*** ({}x int and {}x float) columns'.format(count_int + count_float, count_int, count_float))
    printmd('- There are ***{} categorical*** columns'.format(count_object))
    printmd('- There are ***{} dummy*** columns'.format(len(dummy_list)))
    
    print('___________________________________')
    printmd('- There are ***{} missing values*** in total in the dataset'.format(df.isnull().values.sum()))
    
    text_for_readme += '- There are ***{} numerical*** ({}x int and {}x float) columns\n'.format(count_int + count_float, count_int, count_float)
    text_for_readme += '- There are ***{} categorical*** columns\n'.format(count_object)
    text_for_readme += '- There are ***{} dummy*** columns\n'.format(len(dummy_list))
    text_for_readme += '- There are ***{} missing values*** in total in the dataset\n'.format(df.isnull().values.sum())
    
    with open('text_for_readme.txt', 'w') as f:
        f.write(text_for_readme)
        
get_dataframe_summary(movies, cat_list=['movie_id', 'movie', 'genre'], dummy_list=[])

### DataFrame Overview

- Dataset with 35479 observations and 3 columns

___________________________________


- **Numerical** columns:

,column_name,type,min,max,number NaN


...

- There are ***0 numerical*** (0x int and 0x float) columns

- There are ***3 categorical*** columns

- There are ***0 dummy*** columns

___________________________________


- There are ***249 missing values*** in total in the dataset

In [10]:
get_dataframe_summary(reviews, cat_list=['user_id','movie_id', 'timestamp'], dummy_list=[])

### DataFrame Overview

- Dataset with 863866 observations and 4 columns

___________________________________


- **Numerical** columns:

,column_name,type,min,max,number NaN
0,rating,int64,0,10,0


...

- There are ***1 numerical*** (1x int and 0x float) columns

- There are ***3 categorical*** columns

- There are ***0 dummy*** columns

___________________________________


- There are ***0 missing values*** in total in the dataset

In [11]:
reviews['rating'].max()

10

In [12]:
# Use your findings to match each variable to the correct statement in the dictionary

dict_sol1 = {
'The number of movies in the dataset': movies.shape[0],
'The number of ratings in the dataset': reviews.shape[0],
'The number of different genres': len(movies['genre'].value_counts()),
'The number of unique users in the dataset': len(reviews['user_id'].unique()),
'The number missing ratings in the reviews dataset': reviews.isnull().sum()['rating'],
'The average rating given across all ratings': reviews['rating'].mean(),
'The minimum rating given across all ratings': reviews['rating'].min(),
'The maximum rating given across all ratings': reviews['rating'].max()
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea
for key, value in dict_sol1.items():
    print(key + ' --- ' +  str(value))

The number of movies in the dataset --- 35479
The number of ratings in the dataset --- 863866
The number of different genres --- 2736
The number of unique users in the dataset --- 67353
The number missing ratings in the reviews dataset --- 0
The average rating given across all ratings --- 7.31587769399
The minimum rating given across all ratings --- 0
The maximum rating given across all ratings --- 10


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

## Data Cleaning movies

#### Pull the date from the title and create new column

In [13]:
movies['date'] = movies['movie'].apply(lambda x : re.search(r"\(([0-9]+)\)", x).group(1))
movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [14]:
movies.dtypes

movie_id    object
movie       object
genre       object
date        object
dtype: object

#### Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)

In [15]:
def parse_values(x):
    x = int(x)
    if x in range(1800,1899):
       return 1800
    elif x in range(1900, 1999):
       return 1900
    else:
       return 2000

movies['century'] = movies['date'].apply(parse_values)

movies.sort_values(by='date').head()

,movie_id,movie,genre,date,century
22963,2221420,Sallie Gardner at a Gallop (1878),Short,1878,1800
13515,0392728,Roundhay Garden Scene (1888),Documentary|Short,1888,1800
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1800
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1800
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1800


In [16]:
movies = movies.join(pd.get_dummies(movies['date'].astype(int) // 100).add_suffix('00s'))
movies.head()

,movie_id,movie,genre,date,century,1800s,1900s,2000s
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1800,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1800,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1800,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1800,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1800,1,0,0


#### Dummy column the genre with 1's and 0's

In [17]:
movies['genre_cleaned'] = movies['genre'].apply(lambda x: str(x).split('|')[0])
display(movies.head())
movies
display(movies['genre_cleaned'].value_counts())
display(len(movies['genre_cleaned'].value_counts()))

movies_new = movies.join(pd.get_dummies(movies['genre_cleaned']))

display(movies_new.head())
printmd('#### Shape of movies_one_hot' + str(movies_new.shape))

,movie_id,movie,genre,date,century,1800s,1900s,2000s,genre_cleaned
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1800,1,0,0,Documentary
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1800,1,0,0,Documentary
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1800,1,0,0,Documentary
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1800,1,0,0,nan
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1800,1,0,0,Short


Drama          8370
Comedy         8286
Action         4849
Documentary    3337
Crime          2179
Horror         2136
Adventure      1281
Animation      1250
Biography      1087
Short           908
Thriller        419
nan             249
Romance         198
Fantasy         188
Mystery         187
Family          142
Sci-Fi          127
Western          98
Musical          74
Music            33
History          24
Film-Noir        14
Sport            14
War              14
Adult             7
News              4
Talk-Show         2
Reality-TV        1
Game-Show         1
Name: genre_cleaned, dtype: int64

29

,movie_id,movie,genre,date,century,1800s,1900s,2000s,genre_cleaned,Action,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,nan
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1800,1,0,0,Documentary,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1800,1,0,0,Documentary,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1800,1,0,0,Documentary,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1800,1,0,0,nan,0,...,0,0,0,0,0,0,0,0,0,1
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1800,1,0,0,Short,0,...,0,0,0,1,0,0,0,0,0,0


#### Shape of movies_one_hot(35479, 38)

## Data cleaning reviews

#### Create a date out of time stamp

In [18]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [23]:
reviews['date'] = reviews['timestamp'].apply(lambda x: datetime.fromtimestamp(int(x)))
reviews['month'] = reviews['date'].dt.month
reviews['year'] = reviews['date'].dt.year


reviews_new = reviews.join(pd.get_dummies(reviews['year'], prefix='year'))
reviews_new = reviews_new.join(pd.get_dummies(reviews_new['month'], prefix='month'))

reviews_new.head()



,user_id,movie_id,rating,timestamp,date,month,year,year_2013,year_2014,year_2015,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,0114508,8,1381006850,2013-10-05 21:00:50,10,2013,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,0208092,5,1586466072,2020-04-09 21:01:12,4,2020,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2,0358273,9,1579057827,2020-01-15 03:10:27,1,2020,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,10039344,5,1578603053,2020-01-09 20:50:53,1,2020,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,6751668,9,1578955697,2020-01-13 22:48:17,1,2020,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year

In [21]:
movies['genre'].value_counts()

Drama                                                             3602
Comedy                                                            2091
Documentary                                                       1443
Comedy|Drama                                                      1371
Drama|Romance                                                     1199
Comedy|Drama|Romance                                               988
Comedy|Romance                                                     846
Horror                                                             822
Horror|Thriller                                                    563
Drama|Thriller                                                     503
Crime|Drama|Thriller                                               480
Crime|Drama                                                        434
Thriller                                                           383
Horror|Mystery|Thriller                                            298
Action